In [6]:
import pandas as pd
import pymysql

In [90]:
city2province = {
    "深圳": "广东",
    "广州": "广东",
    "成都": "四川",
    "北京": "北京",
    "上海": "上海",
    "青岛": "山东",
    "南京": "江苏",
    "杭州": "浙江"
}

experienceList = ["在校/应届", "经验不限", "1年以内", "1-3年", "3-5年", "5-10年", "10年以上"]
educationList = ["初中及以下", "中专/中技", "高中", "大专", "本科", "硕士", "博士", "学历不限"]
languageList = [    # 职位列表，要搜索的职位
    'Java', # 100101
    "C-C++", # 100102
    'PHP', # 100103
    'Python', # 100109
    "C#", # 100106
    'Golang', # 100116
    'JavaScript',
    'Swift',
    'Ruby',
    'Kotlin' 
]

In [57]:
# 字符串中筛选出薪资情况
def get_min_max_salary(salary):
    min_salary = salary.split("·")[0].split("-")[0]
    max_salary = salary.split("·")[0].split("-")[1]
    unit = salary.split("·")[0].split("-")[1]

    if("K" in unit): 
        min_salary = int(min_salary) * 1000
        max_salary = int(max_salary.split("K")[0]) * 1000
    elif("元" in unit):
        min_salary = int(min_salary)
        max_salary = int(max_salary.split("元")[0])
        if("天" in unit):
            min_salary = min_salary * 20
            max_salary = max_salary * 20
        if("周" in unit):
            min_salary = min_salary * 4
            max_salary = max_salary * 4
#     print(unit, min_salary, max_salary)
    return min_salary, max_salary

In [123]:
# 清洗数据
def wash_xlsx(xlsx_path, city, language, base):
    df = pd.read_excel(xlsx_path)
# id, province, city, company, scale, language, job, min_salary, max_salary, experience, education, create_time, update_time
    result = [] 
    for row_index, row in df.iterrows(): 
        id = row_index + base
        province = city2province[city]
        city = city
        company = row["公司"]
        scale = row["公司规模"]
        language = language
        job = row["岗位"]
        min_salary, max_salary = get_min_max_salary(row["薪资"])
        experience = row["经验要求"] 
        education = row["学历要求"] 
        if(education not in educationList or experience not in experienceList): # 说明是实习
            education = "本科"
            experience = "在校/应届"
#             print("Error", city, language, row_index)
#             if(education not in educationList): print(education)
#             if(experience not in experienceList): print(experience)
#             continue
        create_time = "2023-11-24 12:00:00"
        update_time = "2023-11-24 12:00:00"
        result.append((
            id, province, city, company, 
            scale, language, job, min_salary, max_salary, 
            experience, education, create_time, update_time
        ))
#         print(result[-1])
    return result
# data = wash_xlsx("./新版/Swift/深圳_Swift.xlsx", "深圳", "Swift")

In [126]:
data = []
for language in languageList:
    for city in city2province.keys():
#         print(len(data), city, language)
        data = data + wash_xlsx(
            "./新版/{}/{}_{}.xlsx".format(language, city, language), 
            city, 
            language, 
            len(data)
        )

# MYSQL 存储

In [127]:
connection = pymysql.connect(
    host='172.19.241.8',  # 数据库主机名
    port=3306,               # 数据库端口号，默认为3306
    user='root',             # 数据库用户名
    passwd='000000',         # 数据库密码
    db='recuritment',               # 数据库名称
    charset='utf8'           # 字符编码
)

# 创建游标对象
cursor = connection.cursor()

In [129]:
def save_data2db(cursor, conn, data):
    sql = """
        INSERT INTO `recurit_info`(
            `id`, `province`, `city`, `company`, 
            `scale`, `language`, `job`, `min_salary`, `max_salary`, 
            `experience`, `education`, `creation_time`, `update_time`
        ) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    try:
        cursor.executemany(sql, data)
        conn.commit()
    except pymysql.Error as e:
        print(e)
        conn.rollback()
save_data2db(cursor, connection, data)